In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# File paths
input_file_path = "/content/drive/MyDrive/M.S. BUSINESS ANALYTICS/MIS612/CO-OP SURVEY GROUP PROJECT/Objective 2/newwork file.xlsx"  # Input file path
output_file_path = "/content/drive/MyDrive/M.S. BUSINESS ANALYTICS/MIS612/CO-OP SURVEY GROUP PROJECT/Objective 2/Objective_2_Findings.xlsx"  # Output file path

# Download required NLTK data
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('punkt', quiet=True)

# Enhanced improvement categories with expanded synonyms
improvement_categories = {
    'In_Person_Work': [
        'person', 'office', 'remote', 'hybrid', 'physical', 'location',
        'site', 'building', 'workplace', 'onsite', 'facility', 'face',
        'virtual', 'inperson', 'workspace', 'presence', 'commute',
        'premises', 'campus', 'branch', 'offline', 'online', 'distance',
        'work from home', 'wfh', 'desk', 'cubicle', 'headquarter', 'hq',
        'return to office', 'rto', 'in-office', 'face to face', 'face-to-face'
    ],
    'Project_Tasks': [
        'project', 'task', 'assignment', 'work', 'responsibility',
        'duty', 'role', 'job', 'workload', 'busy', 'assignment',
        'duties', 'projects', 'tasks', 'variety', 'activities',
        'opportunities', 'more work', 'additional', 'new',
        'hands on', 'hands-on', 'deliverable', 'action', 'objective',
        'goal', 'initiative', 'mission', 'operation', 'function',
        'undertaking', 'engagement', 'campaign', 'program', 'venture',
        'meaningful work', 'substantial', 'significant', 'important'
    ],
    'Team_Interaction': [
        'team', 'people', 'coworker', 'colleague', 'staff', 'member',
        'group', 'others', 'peer', 'social', 'interaction', 'together',
        'collaborative', 'collaboration', 'communicate', 'interact',
        'coworkers', 'colleagues', 'teammates', 'cooperation',
        'network', 'networking', 'bond', 'connect', 'relationship',
        'community', 'fellowship', 'partnership', 'alliance', 'crew',
        'squad', 'unit', 'coordination', 'synergy', 'age', 'younger',
        'older', 'social interaction', 'lunch', 'events', 'gathering'
    ],
    'Learning_Development': [
        'learning', 'training', 'development', 'learn', 'skill',
        'knowledge', 'education', 'growth', 'experience', 'mentor',
        'teaching', 'guidance', 'learn', 'develop', 'improve',
        'understand', 'exposure', 'opportunity', 'career',
        'workshop', 'seminar', 'course', 'certification', 'study',
        'practice', 'instruction', 'coaching', 'tutoring', 'shadowing',
        'apprenticeship', 'internship', 'professional development',
        'skill building', 'expertise', 'competency', 'capability',
        'qualification', 'advancement', 'progress', 'evolve'
    ],
    'Nothing': [
        'nothing', 'none', 'na', 'n/a', 'already', 'perfect', 'great',
        'satisfied', 'enjoyed', 'loved', 'fantastic', 'excellent',
        'good', 'fine', 'ok', 'okay', 'sufficient', 'adequate',
        'complete', 'thorough', 'comprehensive', 'enough', 'satisfied',
        'happy', 'content', 'pleased', 'no', 'not', 'cant think'
    ],
    'Other': []  # Will be calculated based on no matches in other categories
}

def preprocess_text(text):
    """Preprocess text to clean and prepare for analysis."""
    if pd.isna(text) or text.lower() in ['na', 'n/a', '-', 'nothing', 'none', 'idk', 'err:509', '#name?']:
        return ''
    text = str(text).lower()
    text = re.sub(r'[^\w\s]', '', text)
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [t for t in tokens if t not in stop_words]
    lemmatizer = WordNetLemmatizer()
    return ' '.join(lemmatizer.lemmatize(t) for t in tokens)

def analyze_text(response):
    """Analyze the preprocessed text and classify into binary categories."""
    analysis = {category: int(any(k in response for k in keywords))
                for category, keywords in improvement_categories.items() if category != 'Other'}
    # Add the "Other" column: 1 if all categories are 0
    analysis['Other'] = int(all(value == 0 for value in analysis.values()))
    return pd.Series(analysis)

def analyze_responses(file_path, skip_rows=2):
    """Main function to analyze responses from an Excel file."""
    # Load the Excel file
    df = pd.read_excel(file_path, skiprows=skip_rows)

    # Set the correct column names
    df.columns = [
        'Responder ID',
        'Terms',
        'Coll',
        'Major',
        'Conc',
        'Class BOT',
        'Citizenship',
        'Coop #',
        'What would have made this job more interesting to you?'
    ]

    # Preprocess responses
    processed_responses = df['What would have made this job more interesting to you?'].apply(preprocess_text)

    # Analyze text and create binary analysis matrix
    binary_df = processed_responses.apply(analyze_text)

    # Combine original DataFrame with binary analysis results (exclude processed responses)
    result_df = pd.concat([df.drop(columns=['What would have made this job more interesting to you?']), binary_df], axis=1)

    return result_df

if __name__ == "__main__":
    # Analyze responses
    result_df = analyze_responses(input_file_path)

    # Save results (binary columns only)
    result_df.to_excel(output_file_path, index=False)
    print(f"\nData has been saved to {output_file_path}")



Data has been saved to /content/drive/MyDrive/M.S. BUSINESS ANALYTICS/MIS612/CO-OP SURVEY GROUP PROJECT/Objective 2/Objective_2_Findings.xlsx
